How to create a trusted root CA?

Warning: Creating a trusted CA greatly increases your computers attack surface.  This should not be used in a production environment or where PCI (payment card information) or PMI (patient medical information) is held.
https://security.stackexchange.com/questions/44509/what-stops-a-hacker-from-installing-its-own-ca-certificate

Warning: The password is also set to x, this is not secure.  Passwords should be provided by the startup process, and clear when the script is finished.


In [9]:
ROOT=/Users/briancullinan/jupytangular2/Utilities/ca

# prepare root CA
mkdir $ROOT

cd $ROOT
mkdir certs crl newcerts private
chmod 700 private
touch index.txt
echo 1000 > serial

echo "
# OpenSSL root CA configuration file.

[ ca ]
# man ca
default_ca = CA_default

[ CA_default ]
# Directory and file locations.
dir               = /Users/briancullinan/jupytangular2/Utilities/ca
certs             = \$dir/certs
crl_dir           = \$dir/crl
new_certs_dir     = \$dir/newcerts
database          = \$dir/index.txt
serial            = \$dir/serial
RANDFILE          = \$dir/private/.rand

# The root key and root certificate.
private_key       = \$dir/private/ca.key.pem
certificate       = \$dir/certs/ca.cert.pem

# For certificate revocation lists.
crlnumber         = \$dir/crlnumber
crl               = \$dir/crl/ca.crl.pem
crl_extensions    = crl_ext
default_crl_days  = 30

# SHA-1 is deprecated, so use SHA-2 instead.
default_md        = sha256

name_opt          = ca_default
cert_opt          = ca_default
default_days      = 375
preserve          = no
policy            = policy_strict

[ policy_strict ]
# The root CA should only sign intermediate certificates that match.
# See the POLICY FORMAT section of man ca.
countryName             = match
stateOrProvinceName     = match
organizationName        = match
organizationalUnitName  = optional
commonName              = supplied
emailAddress            = optional

[ policy_loose ]
# Allow the intermediate CA to sign a more diverse range of certificates.
# See the POLICY FORMAT section of the ca man page.
countryName             = optional
stateOrProvinceName     = optional
localityName            = optional
organizationName        = optional
organizationalUnitName  = optional
commonName              = supplied
emailAddress            = optional

[ req ]
# Options for the req tool
default_bits        = 2048
distinguished_name  = req_distinguished_name
string_mask         = utf8only

# SHA-1 is deprecated, so use SHA-2 instead.
default_md          = sha256

# Extension to add when the -x509 option is used.
x509_extensions     = v3_ca

[ req_distinguished_name ]
# See <https://en.wikipedia.org/wiki/Certificate_signing_request>.
countryName                     = Country Name (2 letter code)
stateOrProvinceName             = State or Province Name
localityName                    = Locality Name
0.organizationName              = Organization Name
organizationalUnitName          = Organizational Unit Name
commonName                      = Common Name
emailAddress                    = Email Address

# Optionally, specify some defaults.
countryName_default             = GB
stateOrProvinceName_default     = England
localityName_default            =
0.organizationName_default      = Alice Ltd
organizationalUnitName_default  =
emailAddress_default            =

[ v3_ca ]
# Extensions for a typical CA (man x509v3_config).
subjectKeyIdentifier = hash
authorityKeyIdentifier = keyid:always,issuer
basicConstraints = critical, CA:true
keyUsage = critical, digitalSignature, cRLSign, keyCertSign

[ v3_intermediate_ca ]
# Extensions for a typical intermediate CA (man x509v3_config).
subjectKeyIdentifier = hash
authorityKeyIdentifier = keyid:always,issuer
basicConstraints = critical, CA:true, pathlen:0
keyUsage = critical, digitalSignature, cRLSign, keyCertSign

[ usr_cert ]
# Extensions for client certificates (man x509v3_config).
basicConstraints = CA:FALSE
nsCertType = client, email
nsComment = \"OpenSSL Generated Client Certificate\"
subjectKeyIdentifier = hash
authorityKeyIdentifier = keyid,issuer
keyUsage = critical, nonRepudiation, digitalSignature, keyEncipherment
extendedKeyUsage = clientAuth, emailProtection

[ server_cert ]
# Extensions for server certificates (man x509v3_config).
basicConstraints = CA:FALSE
nsCertType = server
nsComment = \"OpenSSL Generated Server Certificate\"
subjectKeyIdentifier = hash
authorityKeyIdentifier = keyid,issuer
keyUsage = critical, digitalSignature, nonRepudiation, keyEncipherment, dataEncipherment
extendedKeyUsage = serverAuth
subjectAltName = @alt_names

[alt_names]
IP.1 = 127.0.0.1
DNS.1 = localhost

[ crl_ext ]
# Extension for CRLs (man x509v3_config).
authorityKeyIdentifier=keyid:always

[ ocsp ]
# Extension for OCSP signing certificates (man ocsp).
basicConstraints = CA:FALSE
subjectKeyIdentifier = hash
authorityKeyIdentifier = keyid,issuer
keyUsage = critical, digitalSignature
extendedKeyUsage = critical, OCSPSigning

" > $ROOT/openssl.cnf

# create root key
cd $ROOT
openssl genrsa -aes256 -passout pass:x -out private/ca.key.pem 4096
chmod 700 private/ca.key.pem
# generate root certificate
cd $ROOT
openssl req -config openssl.cnf \
      -passin pass:x \
      -key private/ca.key.pem \
      -new -x509 -days 7300 -sha256 -extensions v3_ca \
      -subj "/C=US/ST=Who/L=Is/O=John/OU=Galt/CN=localhost" \
      -out certs/ca.cert.pem
chmod 744 certs/ca.cert.pem
# verify root ca
openssl x509 -noout -text -in certs/ca.cert.pem



mkdir: /Users/briancullinan/jupytangular2/Utilities/ca: File exists
Generating RSA private key, 4096 bit long modulus
...................++
...................++
e is 65537 (0x10001)
Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            fb:94:5d:fc:dd:82:4d:ae
    Signature Algorithm: sha256WithRSAEncryption
        Issuer: C=US, ST=Who, L=Is, O=John, OU=Galt, CN=localhost
        Validity
            Not Before: Sep 23 21:29:35 2017 GMT
            Not After : Sep 18 21:29:35 2037 GMT
        Subject: C=US, ST=Who, L=Is, O=John, OU=Galt, CN=localhost
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                Public-Key: (4096 bit)
                Modulus:
                    00:f7:80:f4:c2:2a:fb:d5:cb:2b:5b:01:4a:04:62:
                    40:10:bb:02:2e:f3:70:ad:a1:32:71:39:6a:6f:eb:
                    a9:38:39:bd:3b:e7:a4:f7:45:97:a5:69:15:3b:76:
                    0c:1f:fc:fa:db:6b:c7:1e:4c:ce:7a:43:ba:58:dc:
    

Create an intermediate CA



In [10]:
ROOT=/Users/briancullinan/jupytangular2/Utilities/ca

# prepare intermediate CA
mkdir $ROOT/intermediate
cd $ROOT/intermediate
mkdir certs crl csr newcerts private
chmod 700 private
touch index.txt
echo 1000 > serial
echo 1000 > $ROOT/intermediate/crlnumber

# create intermediate configuration
cp $ROOT/openssl.cnf $ROOT/intermediate/openssl.cnf
echo "
[ CA_default ]
dir             = /Users/briancullinan/jupytangular2/Utilities/ca/intermediate
private_key     = \$dir/private/intermediate.key.pem
certificate     = \$dir/certs/intermediate.cert.pem
crl             = \$dir/crl/intermediate.crl.pem
policy          = policy_loose
" >> $ROOT/intermediate/openssl.cnf

# intermediate private CA key
cd $ROOT
openssl genrsa -aes256 -passout pass:x \
      -out intermediate/private/intermediate.key.pem 4096
# create a intermediate csr
cd $ROOT
openssl req -config intermediate/openssl.cnf -new -sha256 \
      -passin pass:x \
      -subj "/C=US/ST=Who/L=Is/O=John/OU=Galt/CN=localhost" \
      -key intermediate/private/intermediate.key.pem \
      -out intermediate/csr/intermediate.csr.pem
# sign the intermediate cert with root CA
cd $ROOT

openssl ca -config openssl.cnf -extensions v3_intermediate_ca \
      -days 3650 -notext -md sha256 \
      -passin pass:x \
      -batch \
      -in intermediate/csr/intermediate.csr.pem \
      -out intermediate/certs/intermediate.cert.pem

chmod 744 intermediate/certs/intermediate.cert.pem
# verify intermediate CA
openssl x509 -noout -text \
      -in intermediate/certs/intermediate.cert.pem
      
openssl verify -CAfile certs/ca.cert.pem \
      intermediate/certs/intermediate.cert.pem
      
# create certificate chain
cat intermediate/certs/intermediate.cert.pem \
      certs/ca.cert.pem > intermediate/certs/ca-chain.cert.pem
chmod 744 intermediate/certs/ca-chain.cert.pem



Generating RSA private key, 4096 bit long modulus
.....................................................................................................++
......................++
e is 65537 (0x10001)
Using configuration from openssl.cnf
Check that the request matches the signature
Signature ok
Certificate Details:
        Serial Number: 4096 (0x1000)
        Validity
            Not Before: Sep 23 21:29:38 2017 GMT
            Not After : Sep 21 21:29:38 2027 GMT
        Subject:
            countryName               = US
            stateOrProvinceName       = Who
            organizationName          = John
            organizationalUnitName    = Galt
            commonName                = localhost
        X509v3 extensions:
            X509v3 Subject Key Identifier: 
                F8:E1:D0:75:FF:B5:16:63:F9:A6:B9:3E:E4:D1:6E:EF:E9:B0:18:AA
            X509v3 Authority Key Identifier: 
                keyid:5D:4E:0A:1D:31:74:2D:9F:98:2F:15:A0:6D:76:DE:6C:08:AE:65:4A

            

How to generate a self-signed cert?



In [11]:
ROOT=/Users/briancullinan/jupytangular2/Utilities/ca

# create a client key
cd $ROOT
openssl genrsa -aes256 -passout pass:x \
      -out intermediate/private/localhost.key.pem 2048
chmod 700 intermediate/private/localhost.key.pem

# create a client csr
cd $ROOT
openssl req -config intermediate/openssl.cnf \
      -passin pass:x \
      -subj "/C=US/ST=Who/L=Is/O=John/OU=Galt/CN=localhost" \
      -key intermediate/private/localhost.key.pem \
      -new -sha256 -out intermediate/csr/localhost.csr.pem
# sign the client certificate with the intermediate CA
cd $ROOT
openssl ca -config intermediate/openssl.cnf \
      -passin pass:x \
      -extensions server_cert -days 375 -notext -md sha256 \
      -batch \
      -in intermediate/csr/localhost.csr.pem \
      -out intermediate/certs/localhost.cert.pem
chmod 744 intermediate/certs/localhost.cert.pem
# verify client certificate
openssl x509 -noout -text \
      -in intermediate/certs/localhost.cert.pem
openssl verify -CAfile intermediate/certs/ca-chain.cert.pem \
      intermediate/certs/localhost.cert.pem      


#openssl genrsa -des3 -passout pass:x -out server.pass.key 2048
#openssl rsa -passin pass:x -in server.pass.key -out server.key
#rm server.pass.key
#openssl req -new -key server.key -out server.csr
#openssl x509 -req -sha256 -days 365 -in server.csr -signkey server.key -out server.crt
#security add-trusted-cert -d -r trustRoot -k /Users/briancullinan/Library/Keychains/login.keychain /Users/briancullinan/jupytangular2/Utilities/ca/certs/ca.cert.pem
#security add-trusted-cert -d -r trustRoot -k /Users/briancullinan/Library/Keychains/login.keychain /Users/briancullinan/jupytangular2/Utilities/ca/intermediate/certs/localhost.cert.pem


Generating RSA private key, 2048 bit long modulus
.................+++
........................................................................................................+++
e is 65537 (0x10001)
Using configuration from intermediate/openssl.cnf
Check that the request matches the signature
Signature ok
Certificate Details:
        Serial Number: 4097 (0x1001)
        Validity
            Not Before: Sep 23 21:29:40 2017 GMT
            Not After : Oct  3 21:29:40 2018 GMT
        Subject:
            countryName               = US
            stateOrProvinceName       = Who
            localityName              = Is
            organizationName          = John
            organizationalUnitName    = Galt
            commonName                = localhost
        X509v3 extensions:
            X509v3 Basic Constraints: 
                CA:FALSE
            Netscape Cert Type: 
                SSL Server
            Netscape Comment: 
                OpenSSL Generated Server Certifica